In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load FER2013
data = pd.read_csv('fer2013.csv')

# Filter only 'happy' and 'sad' classes (3 = happy, 4 = sad)
data = data[(data['emotion'] == 3) | (data['emotion'] == 4)]

# Remap labels: 3 → 0 (happy), 4 → 1 (sad)
data['emotion'] = data['emotion'].map({3: 0, 4: 1})

# Process pixels
pixels = data['pixels'].tolist()
faces = np.array([np.fromstring(p, sep=' ') for p in pixels])
faces = faces.reshape(-1, 48, 48, 1).astype('float32') / 255.0

# Labels
labels = data['emotion'].values
labels = to_categorical(labels, num_classes=2)  # One-hot encode

# Split
X_train, X_val, y_train, y_val = train_test_split(faces, labels, test_size=0.2, random_state=42)


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # Binary classification (happy/sad)
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       409,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 428,610 (1.64 MB)

 Trainable params: 428,610 (1.64 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=64,
    epochs=20  # should be enough for this task
)


Epoch 1/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 38s 190ms/step - accuracy: 0.6210 - loss: 0.6541 - val_accuracy: 0.7040 - val_loss: 0.5468
Epoch 2/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 40s 184ms/step - accuracy: 0.7377 - loss: 0.5212 - val_accuracy: 0.7598 - val_loss: 0.4989
Epoch 3/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 41s 186ms/step - accuracy: 0.7724 - loss: 0.4672 - val_accuracy: 0.7797 - val_loss: 0.4538
Epoch 4/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 40s 183ms/step - accuracy: 0.7875 - loss: 0.4277 - val_accuracy: 0.7873 - val_loss: 0.4350
Epoch 5/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 39s 175ms/step - accuracy: 0.8074 - loss: 0.4028 - val_accuracy: 0.8049 - val_loss: 0.4135
Epoch 6/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 40s 172ms/step - accuracy: 0.8257 - loss: 0.3734 - val_accuracy: 0.8125 - val_loss: 0.3994
Epoch 7/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 42s 178ms/step - accuracy: 0.8406 - loss: 0.3421 - val_accuracy: 0.8149 - val_loss: 0.3929
Epoch 8/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 41s 179ms/step - accuracy: 0.8504 - loss: 0

In [6]:
from keras.saving import save_model
save_model(model, "happy_sad_model.keras")



In [9]:
loss, acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {acc*100:.2f}%, Loss: {loss:.4f}")


95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.8349 - loss: 0.4032
Validation Accuracy: 84.31%, Loss: 0.3983


In [10]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)

print(confusion_matrix(y_true, y_pred_classes))
print(classification_report(y_true, y_pred_classes, target_names=['Happy', 'Sad']))


95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step
[[1493  260]
 [ 213 1048]]
              precision    recall  f1-score   support

       Happy       0.88      0.85      0.86      1753
         Sad       0.80      0.83      0.82      1261

    accuracy                           0.84      3014
   macro avg       0.84      0.84      0.84      3014
weighted avg       0.84      0.84      0.84      3014

